---
## Lab Number : 0

### Title : *Introduction to Spark* 

### Goal : Spark basics. Getting famililar with RDDs / Dataframes / Transformations and Actions  

### In this Lab we will cover the:

1. Jupyter Notebooks basic intro
2. Using Apache Spark with pySpark API (and possibly other APIs)  
3. Basics of RDDs : transformations and actions
4. Lambda functions
5. More complex RDD actions
6. More complex RDD transformations
7. RDDs Lazy Evaluation 

### References:

1. Jupyter Notebooks : https://try.jupyter.org/
2. Spark API reference : https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD

---


### 1. Jupyter Notebooks

Jupyter Notebooks are a very powerfull tool that allow Scientist , Data Scientist and Engineers  